In [1]:
from __future__ import unicode_literals # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
from google.cloud import vision
from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup


# 네이버 웹툰 썸네일크롤링
 - 장르, 연령, 간략줄거리 -> 데이터프레임

In [2]:
def crawling_naver_webtoon_thumbnail():
    webtoon_info = []
    total_df_info = []
    total_img_count = 0

    domain_url = 'https://comic.naver.com'
    main_url = 'https://comic.naver.com/webtoon/weekday.nhn'

    html = requests.get(main_url).text
    soup = BeautifulSoup(html, 'html.parser')
    webtoon_area = soup.select('#content > div.list_area.daily_all > div.col > div > ul > li > div.thumb > a')

    for each_webtoon in webtoon_area:
        webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
        html = requests.get(webtoon_last_page).text
        soup = BeautifulSoup(html, 'html.parser')
        last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)

        title = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
        title = ', '.join(['&'.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in title.split(',')])
        genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')[0].text
        content = soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')[0].text
        age = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.age')[0].text

        webtoon_info.append({'title': title, 'genre': genre, 'content': content, 'age': age})

        for page_num in range(1, last_page_num + 1):
            webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
            html = requests.get(webtoon_url).text
            soup = BeautifulSoup(html, 'html.parser')

            if not os.path.exists('./imgs/{}'.format(title)):
                os.mkdir('./imgs/{}'.format(title))

            thumbnail_area = soup.find('table', 'viewList')
            thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]

            for each_img_tag in thumbnail_img_list:
                urllib.request.urlretrieve(each_img_tag['src'], './imgs/{}/{}.png'.format(title, title + '_' + '&'.join(each_img_tag['alt'].split('/'))))
                urllib.request.urlretrieve(each_img_tag['src'], './total_img/{}.png'.format(title + '_' + '&'.join(each_img_tag['alt'].split('/'))))
                total_df_info.append({'title': title + '_' + '&'.join(each_img_tag['alt'].split('/')), 'genre': genre, 'content': content, 'age': age})
                total_img_count += 1

    df = pd.DataFrame(webtoon_info, columns=['title', 'genre', 'content', 'age'])
    total_df = pd.DataFrame(total_df_info, columns=['title', 'genre', 'content', 'age'])
    df.to_csv('webtoon_info.csv')
    total_df.to_csv('total_img_info.csv')

In [48]:
webtoon_info = []
total_df_info = []
total_img_count = 0

domain_url = 'https://comic.naver.com'
main_url = 'https://comic.naver.com/webtoon/weekday.nhn'

html = requests.get(main_url).text
soup = BeautifulSoup(html, 'html.parser')
webtoon_area = soup.select('#content > div.list_area.daily_all > div.col > div > ul > li > div.thumb > a')

for each_webtoon in webtoon_area:
    webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
    html = requests.get(webtoon_last_page).text
    soup = BeautifulSoup(html, 'html.parser')
    last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)

    title = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
    title = ', '.join(['&'.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in title.split(',')])
    genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')[0].text
    content = soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')[0].text
    age = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.age')[0].text

    webtoon_info.append({'title': title, 'genre': genre, 'content': content, 'age': age})

    for page_num in range(1, last_page_num + 1):
        webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
        html = requests.get(webtoon_url).text
        soup = BeautifulSoup(html, 'html.parser')

        if not os.path.exists('./imgs/{}'.format(title)):
            os.mkdir('./imgs/{}'.format(title))

        thumbnail_area = soup.find('table', 'viewList')
        thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]

        for each_img_tag in thumbnail_img_list:
            urllib.request.urlretrieve(each_img_tag['src'], './imgs/{}/{}.png'.format(title, title + '_' + '&'.join(each_img_tag['alt'].split('/'))))
            urllib.request.urlretrieve(each_img_tag['src'], './total_img/{}.png'.format(title + '_' + '&'.join(each_img_tag['alt'].split('/'))))
            total_df_info.append({'title': title + '_' + '&'.join(each_img_tag['alt'].split('/')), 'genre': genre, 'content': content, 'age': age})
            total_img_count += 1

df = pd.DataFrame(webtoon_info, columns=['title', 'genre', 'content', 'age'])
total_df = pd.DataFrame(total_df_info, columns=['title', 'genre', 'content', 'age'])
df.to_csv('webtoon_info.csv')
total_df.to_csv('total_img_info.csv')


In [41]:
total_img_count

27860